# Experiment Introduction

Background, 
In order to do ablation experiment, we have diffculty like
1. most of them are same, we have to repeat many times.
2. too many model, config, code hard to manage and save. too messy


For all kind of SSL training workflow, we have to define the hyperparameters includes 4 aspect,
Dataset
Model
Training
Saving COnfig



How to Use this Experiment?



# Library

In [14]:
# Import config
import sys 
sys.path.append("C:/Users/isxzl/OneDrive/Code/AutoSSL")
import torch
import yaml
from torchvision.transforms import RandomRotation,GaussianBlur,ColorJitter
from autoSSL.evaluate import eval_KNN,eval_linear,eval_KNNplot,pipe_collate
from autoSSL.models import BarlowTwins, BYOL, MoCo, SimCLR, SimSiam, VICReg ,pipe_model 
from autoSSL.utils import embedding_feature,ck_callback,dict2transformer,join_dir,ContinuousCSVLogger  
from autoSSL.data import PipeDataset
from autoSSL.train import Trainer
 


# Import Global Baseline Config

In [15]:
# Load the YAML file
with open('global.yaml', 'r') as file:
    global_config = yaml.safe_load(file)

# Write your experiment notebook name here
global_config["experiment"]="backbone VS model"     
    
# Define global view function
global_SSL_augmentation = {
    'RandomResizedCrop': {'size': (global_config["input_size"], global_config["input_size"])},
    'RandomApply':{'transforms':[RandomRotation(degrees=90)], 'p':0.8},
    'RandomHorizontalFlip': {'p': 0.5},
    'RandomVerticalFlip':  {'p':0.5},
    'RandomApply':{'transforms': [ColorJitter(brightness=0.04,contrast=0.04,saturation=0.02,hue=0.01)], 'p':0.8},
    'RandomGrayscale' :{'p':0.2},
    'RandomSolarize':{'threshold':128, 'p':0.1},
    'RandomApply':{'transforms':[GaussianBlur(kernel_size=3,sigma=(0.2, 2))],'p':0.8},
    'ToTensor': {},
    'Normalize': {"mean": [0.485, 0.456, 0.406], "std": [0.229, 0.224, 0.225]}
}


# Train Model

## vicreg

In [26]:
import GPUtil
GPUs = GPUtil.getGPUs()
gpu = GPUs[0]
import torch
print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
print(torch.cuda.memory_summary ())

GPU RAM Free: 16283MB | Used: 7872MB | Util  32% | Total 24564MB
|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |    943 MiB |  13048 MiB |  60598 GiB |  60597 GiB |
|       from large pool |    768 MiB |  12913 MiB |  60293 GiB |  60292 GiB |
|       from small pool |    174 MiB |    197 MiB |    305 GiB |    305 GiB |
|---------------------------------------------------------------------------|
| Active memory         |    943 MiB |  13048 MiB |  60598 GiB |  60597 GiB |

In [28]:
# "vit_64","mobilenet_v3", "mobilenet_v3_pretrained", "resnet50","efficientnet_b5", "efficientnet_b5_pretrained"
for backbone_name in["resnet18","resnet18_pretrained", "resnet50","resnet50_pretrained"]:

    # MAKE YOUR OWN CONFIG
    config=global_config.copy()
    # Fill the config
    SSL_augmentation=global_SSL_augmentation.copy()
    config["name"]=f"VICReg_backbone_{backbone_name}"
    config["backbone"]=backbone_name
    config["model"]="VICReg"
    config["batch"]=512
    config["prjhead_dim"]=2048
    # THIS IS THE CODE TO LOAD DATASET
    pdata= PipeDataset(config=config,augmentation=dict2transformer(SSL_augmentation,view=config["view"]))
    # THIS IS THE CODE TO LOAD MODEL
    pmodel=pipe_model(config=config)

    # Use this if you want to START a train
    #trainer=Trainer(config, model_mode="start",precision='16-mixed')
    #trainer.fit(pmodel, pdata.dataloader)  
    #trainer=Trainer(config, model_mode="continue", extra_epoch=3,precision='16-mixed')
    #trainer.fit(pmodel, pdata.dataloader,ckpt_path="latest")  
    del pdata
    del pmodel
    del trainer
    torch.cuda.empty_cache()
    # Use this if you want to CONTINUE to train
    

Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 4090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
Restoring states from the checkpoint path at experiment_checkpoints/backbone VS model\VICReg_backbone_resnet18\checkpoints-epoch=07-train_loss=20.23.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name            | Type                      | Params
--------------------------------------------------------------
0 | backbone        | Sequential                | 11.2 M
1 | projection_head | BarlowTwinsProjectionHead | 9.4 M 
2 | crite

Previous max epoch: 8. Continue to train for extra 3 epochs.
Loading checkpoint from: experiment_checkpoints/backbone VS model\VICReg_backbone_resnet18\checkpoints-epoch=07-train_loss=20.23.ckpt
Epoch 10: 100%|███████████████████████████████████████████████████████████████| 98/98 [00:36<00:00,  2.69it/s, v_num=0]

`Trainer.fit` stopped: `max_epochs=11` reached.


Epoch 10: 100%|███████████████████████████████████████████████████████████████| 98/98 [00:36<00:00,  2.68it/s, v_num=0]


Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 4090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
C:\Users\isxzl\anaconda3\envs\AutoGPT\lib\site-packages\pytorch_lightning\callbacks\model_checkpoint.py:612: UserWarning: Checkpoint directory C:\Users\isxzl\OneDrive\Code\AutoSSL\experiment\experiment_checkpoints\backbone VS model\VICReg_backbone_resnet18_pretrained exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
Restoring states from the checkpoint path at experiment_checkpoints/ba

...

`Trainer.fit` stopped: `max_epochs=8` reached.


Epoch 7: 100%|████████████████████████████████████████████████████████████████| 98/98 [00:33<00:00,  2.95it/s, v_num=0]


Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 4090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
C:\Users\isxzl\anaconda3\envs\AutoGPT\lib\site-packages\pytorch_lightning\callbacks\model_checkpoint.py:612: UserWarning: Checkpoint directory C:\Users\isxzl\OneDrive\Code\AutoSSL\experiment\experiment_checkpoints\backbone VS model\VICReg_backbone_resnet50 exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
Restoring states from the checkpoint path at experiment_checkpoints/backbone VS m

Previous max epoch: 5. Continue to train for extra 3 epochs.
Loading checkpoint from: experiment_checkpoints/backbone VS model\VICReg_backbone_resnet50\checkpoints-epoch=02-train_loss=nan-v1.ckpt


RuntimeError: Error(s) in loading state_dict for VICReg:
	size mismatch for projection_head.layers.0.weight: copying a param with shape torch.Size([1024, 2048]) from checkpoint, the shape in current model is torch.Size([2048, 2048]).
	size mismatch for projection_head.layers.1.weight: copying a param with shape torch.Size([1024]) from checkpoint, the shape in current model is torch.Size([2048]).
	size mismatch for projection_head.layers.1.bias: copying a param with shape torch.Size([1024]) from checkpoint, the shape in current model is torch.Size([2048]).
	size mismatch for projection_head.layers.1.running_mean: copying a param with shape torch.Size([1024]) from checkpoint, the shape in current model is torch.Size([2048]).
	size mismatch for projection_head.layers.1.running_var: copying a param with shape torch.Size([1024]) from checkpoint, the shape in current model is torch.Size([2048]).
	size mismatch for projection_head.layers.3.weight: copying a param with shape torch.Size([1024, 

## Config 2

## Config 3

# Evaluation

In [10]:
import os
import glob
import re
import pandas as pd
import numpy as np
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score
from torch.utils.data import DataLoader
from tqdm import tqdm
import torch

def embed(x, embedding_model, device):
    embedding_model.eval()
    embedding_model.to(device)
    x = x.float().to(device)   # remove the unsqueeze operation
    return embedding_model(x).detach().cpu().numpy()


def eval_linear(pipe_data, models, device='cuda', split=None, test=None):
    '''
    Parameters:
    pipe_data (PipeDataset): The whole data as a PipeDataset object
    models (dict/torch.nn.Module): Model dict returned by `pipe_collate` or a single model to create embeddings
    device (str): Device to perform computations on. Default is 'cuda' if available.
    split (float, optional): The ratio of samples to include in the train split.
    test (PipeDataset, optional): The test data as a PipeDataset object
    '''

    # Use split parameter to divide data into train and test if test is not provided
    if split is not None and test is None:
        train_data, test_data = pipe_data.split(split)
    else:
        train_data = pipe_data
        test_data = test

    # Extract features and labels from train and test data
    print("Load the training and testing dataset")
    X_train, y_train = train_data.array[0], train_data.array[1]
    X_test, y_test = test_data.array[0], test_data.array[1]

    if isinstance(models, torch.nn.Module):
       models = {'name': ['model_0'], 'model': [models], 'address': None} 
    if isinstance(models, list):
        models = {'name': ['model_'+str(i) for i in range(0,len(models))], 'model': models, 'address': None}
   
    # Initialize the results list
    results = []

    # Iterate over models and calculate accuracy
    for i, embedding_model in enumerate(tqdm(models['model'])):
        # Get the embeddings for train and test data
        X_train_embedding = [embed(x, embedding_model, device) for x in DataLoader(X_train, batch_size=128)]
        X_train_embedding = np.concatenate(X_train_embedding)

        X_test_embedding = [embed(x, embedding_model, device) for x in DataLoader(X_test, batch_size=128)]
        X_test_embedding = np.concatenate(X_test_embedding)

        if X_test_embedding is None or np.any(np.isnan(X_train_embedding)):
            accuracy = 'model_collapse'
        else:
            # Initialize the SGDClassifier and train it
            clf = SGDClassifier()
            clf.fit(X_train_embedding, y_train)

            # Predict labels for the test data and calculate accuracy
            X_test_predicted = clf.predict(X_test_embedding)
            accuracy = accuracy_score(y_test, X_test_predicted)

        namee=models["name"][i]
        # Append the accuracy to the results
        results.append((namee, accuracy))
        
    if models['address'] is not None:
        df = pd.read_csv(models['address'])
        df['linear_accuracy'] = [result[1] for result in results]
        df.to_csv(models['address'], index=False)
    
    return results

In [11]:

test_augmentation = {
    'RandomResizedCrop': {'size': (global_config["input_size"], global_config["input_size"])},
    'ToTensor': {},
    'Normalize': {"mean": [0.485, 0.456, 0.406], "std": [0.229, 0.224, 0.225]}
}
collate =pipe_collate(address="experiment_checkpoints/backbone VS model/", reg="^VICReg_backbone_.*$")

pdata = PipeDataset(input_dir=global_config["path_to_test_cifar10"],
    augmentation=dict2transformer(test_augmentation,view=1))


Collating the models' (evaluating) information to experiment_checkpoints/backbone VS model/^VICReg_backbone_._$.csv


In [12]:

aaa=eval_linear(pdata, models=collate, device=global_config["device"], split=0.8)


Load the training and testing dataset


100%|████████████████████████████████████████████████████████████████████████████████████| 9/9 [00:54<00:00,  6.02s/it]


In [ ]:
 aaa

[('VICReg_backbone_efficientnet_b5', 'model_collapse'),
 ('VICReg_backbone_efficientnet_b5_pretrained', 0.30684657671164417),
 ('VICReg_backbone_mobilenet_v3', 'model_collapse'),
 ('VICReg_backbone_mobilenet_v3_pretrained', 0.43228385807096453),
 ('VICReg_backbone_resnet18', 0.22938530734632684),
 ('VICReg_backbone_resnet18_pretrained', 0.32233883058470764),
 ('VICReg_backbone_resnet50', 'model_collapse'),
 ('VICReg_backbone_resnet50_pretrained', 0.3363318340829585),
 ('VICReg_backbone_vit_64', 0.1054472763618191)]